In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import joblib
%matplotlib inline
import seaborn as sns

from sklearn import tree
from sklearn.model_selection import train_test_split

In [4]:
X = pd.read_csv('application_train.csv')
X.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
X_ext = X[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'TARGET']]

In [35]:
for function_name in ['nanmin', 'nanmax', 'sum', 'mean', 'var', 'median', 'std', 'nanmedian', 'nanmean', 'min', 'max']:
    X_ext['external_sources_{}'.format(function_name)] = eval('np.{}'.format(function_name))(
        X_ext[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)

<ipython-input-35-7d6a5c537d7b>:2: RuntimeWarning: All-NaN axis encountered
  X_ext['external_sources_{}'.format(function_name)] = eval('np.{}'.format(function_name))(
<ipython-input-35-7d6a5c537d7b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_ext['external_sources_{}'.format(function_name)] = eval('np.{}'.format(function_name))(
C:\Users\asus\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,
<ipython-input-35-7d6a5c537d7b>:2: RuntimeWarning: Mean of empty slice
  X_ext['external_sources_{}'.format(function_name)] = eval('np.{}'.format(function_name))(


In [37]:
X_ext['EXT_SRC_weighted3'] = (X.EXT_SOURCE_1*2+X.EXT_SOURCE_2*3+X.EXT_SOURCE_3*4)/9
X_ext['EXT_SRC_weighted2'] = (X.EXT_SOURCE_1*3+X.EXT_SOURCE_2*4+X.EXT_SOURCE_3*2)/9
X_ext['EXT_SRC_weighted1'] = (X.EXT_SOURCE_1*4+X.EXT_SOURCE_2*2+X.EXT_SOURCE_3*3)/9

<ipython-input-37-aa2eda1f0d00>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_ext['EXT_SRC_weighted3'] = (X.EXT_SOURCE_1*2+X.EXT_SOURCE_2*3+X.EXT_SOURCE_3*4)/9
<ipython-input-37-aa2eda1f0d00>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_ext['EXT_SRC_weighted2'] = (X.EXT_SOURCE_1*3+X.EXT_SOURCE_2*4+X.EXT_SOURCE_3*2)/9
<ipython-input-37-aa2eda1f0d00>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [38]:
X_ext['SK_ID_CURR'] = X['SK_ID_CURR']
X_ext = X_ext.reset_index(drop=True)
X_ext = X_ext.set_index('SK_ID_CURR')

<ipython-input-38-530f4cf14f8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_ext['SK_ID_CURR'] = X['SK_ID_CURR']


In [39]:
X_ext

,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,TARGET,external_sources_nanmin,external_sources_nanmax,external_sources_sum,external_sources_mean,external_sources_var,external_sources_median,external_sources_std,external_sources_nanmedian,external_sources_nanmean,external_sources_min,external_sources_max,EXT_SRC_weighted3,EXT_SRC_weighted2,EXT_SRC_weighted1
SK_ID_CURR,,,,,,,,,,,,,,,,,,
100002,0.083037,0.262949,0.139376,1,0.083037,0.262949,0.485361,0.161787,0.005646,0.139376,0.075139,0.139376,0.161787,0.083037,0.262949,0.168047,0.175517,0.141797
100003,0.311267,0.622246,NaN,0,0.311267,0.622246,0.933513,0.466757,0.024177,NaN,0.155489,0.466757,0.466757,0.311267,0.622246,NaN,NaN,NaN
100004,NaN,0.555912,0.729567,0,0.555912,0.729567,1.285479,0.642739,0.007539,NaN,0.086827,0.642739,0.642739,0.555912,0.729567,NaN,NaN,NaN
100006,NaN,0.650442,NaN,0,0.650442,0.650442,0.650442,0.650442,0.000000,NaN,0.000000,0.650442,0.650442,0.650442,0.650442,NaN,NaN,NaN
100007,NaN,0.322738,NaN,0,0.322738,0.322738,0.322738,0.322738,0.000000,NaN,0.000000,0.322738,0.322738,0.322738,0.322738,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0.145570,0.681632,NaN,0,0.145570,0.681632,0.827203,0.413601,0.071841,NaN,0.268031,0.413601,0.413601,0.145570,0.681632,NaN,NaN,NaN
456252,NaN,0.115992,NaN,0,0.115992,0.115992,0.115992,0.115992,0.000000,NaN,0.000000,0.115992,0.115992,0.115992,0.115992,NaN,NaN,NaN
456253,0.744026,0.535722,0.218859,0,0.218859,0.744026,1.498607,0.499536,0.046621,0.535722,0.215920,0.535722,0.499536,0.218859,0.744026,0.441184,0.534743,0.522681


In [50]:
X_ext = X_ext.drop(columns=['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'])
X_ext

,external_sources_nanmin,external_sources_nanmax,external_sources_sum,external_sources_mean,external_sources_var,external_sources_median,external_sources_std,external_sources_nanmedian,external_sources_nanmean,external_sources_min,external_sources_max,EXT_SRC_weighted3,EXT_SRC_weighted2,EXT_SRC_weighted1
SK_ID_CURR,,,,,,,,,,,,,,
100002,0.083037,0.262949,0.485361,0.161787,0.005646,0.139376,0.075139,0.139376,0.161787,0.083037,0.262949,0.168047,0.175517,0.141797
100003,0.311267,0.622246,0.933513,0.466757,0.024177,NaN,0.155489,0.466757,0.466757,0.311267,0.622246,NaN,NaN,NaN
100004,0.555912,0.729567,1.285479,0.642739,0.007539,NaN,0.086827,0.642739,0.642739,0.555912,0.729567,NaN,NaN,NaN
100006,0.650442,0.650442,0.650442,0.650442,0.000000,NaN,0.000000,0.650442,0.650442,0.650442,0.650442,NaN,NaN,NaN
100007,0.322738,0.322738,0.322738,0.322738,0.000000,NaN,0.000000,0.322738,0.322738,0.322738,0.322738,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0.145570,0.681632,0.827203,0.413601,0.071841,NaN,0.268031,0.413601,0.413601,0.145570,0.681632,NaN,NaN,NaN
456252,0.115992,0.115992,0.115992,0.115992,0.000000,NaN,0.000000,0.115992,0.115992,0.115992,0.115992,NaN,NaN,NaN
456253,0.218859,0.744026,1.498607,0.499536,0.046621,0.535722,0.215920,0.535722,0.499536,0.218859,0.744026,0.441184,0.534743,0.522681


In [51]:
output_file_path = 'ext_source_eng.parquet'

X_ext.to_parquet(output_file_path, index=True)